In [1]:
import numpy as np
import gym
from gym import wrappers

Define the functions to run an episode and evaluate, extract and compute the policy.

In [2]:
def run_episode_return_reward(environment, policy, gamma_value = 1.0, render = False):
    """ Runs an episode and return the total reward """
    obs = environment.reset()
    total_reward = 0
    step_index = 0
    while True:
        if render:
            environment.render()
        obs, reward, done , _ = environment.step(int(policy[obs]))
        total_reward += (gamma_value ** step_index * reward)
        step_index += 1
        if done:
            break
    return total_reward

def evaluate_policy(environment, policy, gamma_value = 1.0, n = 200):
    model_scores = [run_episode_return_reward(environment, policy, gamma_value, False) for _ in range(n)]
    return np.mean(model_scores)

def extract_policy(v, gamma_value = 1.0):
    """ Extract the policy for a given value-function """
    policy = np.zeros(environment.nS)
    for s in range(environment.nS):
        q_sa = np.zeros(environment.nA)
        for a in range(environment.nA):
            q_sa[a] = sum([p * (r + gamma_value * v[s_]) for p, s_, r, _ in  environment.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

def compute_policy_v(environment, policy, gamma_value=1.0):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(environment.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(environment.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma_value * prev_v[s_]) for p, s_, r, _ in environment.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

Define a function to perform policy iteration.

In [3]:
def policy_iteration(environment, gamma_value = 1.0):
    """ Policy-Iteration algorithm """
    # Initialize a random policy
    policy = np.random.choice(environment.nA, size=(environment.nS))  
    max_iterations = 100000
    gamma_value = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(environment, policy, gamma_value)
        new_policy = extract_policy(old_policy_v, gamma_value)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

From here we can run policy iteration on the Frozen Lake environment. 

In [5]:
env_name  = 'FrozenLake-v0'
environment = gym.make(env_name)
optimal_policy = policy_iteration(environment, gamma_value = 1.0)
scores = evaluate_policy(environment, optimal_policy, gamma_value = 1.0)
print('Average scores = ', np.mean(scores))

Policy-Iteration converged at step 5.
Average scores =  0.74
